# Scraping Toronto Postal Codes from Wikipedia

Copied from Assignment: 
- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

<B>Continue with Pt 2 - Combine Postal w Lat/Long</b)

My steps:
1) Identify the Wikipedia page for Postal Codes

2) Create the empty DF w columns

3) Step 7/8 Read in the table and remove \n formatting

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
webcopy = requests.get(url).text
postaldata = BeautifulSoup(webcopy, 'lxml')

In [3]:
# create the new Toronto Dataframe
columnlabel = ['Postalcode','Borough','Neighborhood']
tor_codes = pd.DataFrame(columns = columnlabel)

In [4]:
tor_codes.head()

,Postalcode,Borough,Neighborhood


In [5]:
# read through table to get postcode, borough, neighborhood 
wikicontent = postaldata.find('div', class_='mw-parser-output')
table = wikicontent.table.tbody
postcode = 0
borough = 0
neighborhood = 0

In [6]:
tor_codes.head()

,Postalcode,Borough,Neighborhood


In [7]:
for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text.strip('\n').replace(']','')  # postcode = td.text
            i = i + 1
            # print(postcode,"tim1",i)  #added
        elif i == 1:
            borough = td.text.strip('\n').replace(']','')   # borough1 = td.text
            i = i + 1
            # print(borough,i)  #added
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
            # print(neighborhood,"tim3",i)  #added
    tor_codes = tor_codes.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

In [8]:
tor_codes.head()

,Postalcode,Borough,Neighborhood
0,0,0,0
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [9]:
# clean dataframe 
tor_codes = tor_codes[tor_codes.Borough!='Not assigned']
tor_codes = tor_codes[tor_codes.Borough!= 0]
tor_codes.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,tor_codes.shape[0]):
    if tor_codes.iloc[i][2] == 'Not assigned':
        tor_codes.iloc[i][2] = tor_codes.iloc[i][1]
        i = i+1

In [10]:
tor_codes.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [11]:
df = tor_codes.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
# drop burroughs with none assigned
df = df.dropna()
empty = 'Not assigned'
df = df[(df.Postalcode != empty ) & (df.Borough != empty) & (df.Neighborhood != empty)]

In [12]:
# group neighborhoods with like burroughs
def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))

In [13]:
tor_codes.head(20)

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [15]:
print(tor_codes.shape)

(103, 3)
